In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import copy
import re

import torch
import torch.nn as nn
import torch.utils.data as Data

import torchvision
from torch.utils import model_zoo
import torchvision.transforms as transforms

from PIL import Image

import skimage
from skimage import transform as tf
from skimage import io

In [11]:
# Sample triplets and save paths as texts (No use)
# Cost about 30 minutes to sample this textfile
import os

train_dir = "tiny-imagenet-200/train"
imtp_name = os.listdir(train_dir)

start_time = datetime.datetime.now()
print("Start Time: ", start_time)

for n in range(num_epochs):
    print("Writing Epoch {} Triplet...".format(n+1))
    filename = "ImagePath/Epoch_{}_Triplet.txt".format(n+1)
    file1 = open(filename,"w") 
    for imtpidx in range(num_tp):
        typename = imtp_name[imtpidx]
        imfolder = "/".join([train_dir, typename, "images"])
        #imfolder = os.path.join(train_dir, typename, "images")
        im_name = os.listdir(imfolder)
        for imidx in range(num_im):
            im_qr = "/".join(["HW5",imfolder, im_name[imidx]])
            #im_qr = os.path.join(imfolder, im_name[imidx]) # Save paths as text

            samp_im_ps = np.random.choice(im_name[:imidx]+im_name[imidx+1:])
            im_ps = "/".join(["HW5",imfolder, samp_im_ps])
            #im_ps = os.path.join(imfolder, samp_im_ps)

            imtp_ng = np.random.choice(imtp_name[:imtpidx]+imtp_name[imtpidx+1:]) # Type of trainset
            imfolder_ng = "/".join([train_dir, imtp_ng, "images"])
            #imfolder_ng = os.path.join(train_dir, imtp_ng, "images")
            im_ng_name = os.listdir(imfolder_ng)
            samp_im_ng = np.random.choice(im_ng_name)
            im_ng = "/".join(["HW5",imfolder_ng, samp_im_ng])
            #im_ng = os.path.join(imfolder_ng, samp_im_ng)

            file1.writelines("\t".join([im_qr, im_ps, im_ng, typename, "\n"]))
    file1.close()

end_time = datetime.datetime.now()
print("Cost time", end_time-start_time)

Start Time:  2018-10-18 01:12:48.186049
Writing Epoch 1 Triplet...
Writing Epoch 2 Triplet...
Writing Epoch 3 Triplet...
Writing Epoch 4 Triplet...
Writing Epoch 5 Triplet...
Writing Epoch 6 Triplet...
Writing Epoch 7 Triplet...
Writing Epoch 8 Triplet...
Writing Epoch 9 Triplet...
Writing Epoch 10 Triplet...
Writing Epoch 11 Triplet...
Writing Epoch 12 Triplet...
Writing Epoch 13 Triplet...
Writing Epoch 14 Triplet...
Writing Epoch 15 Triplet...
Writing Epoch 16 Triplet...
Writing Epoch 17 Triplet...
Writing Epoch 18 Triplet...
Writing Epoch 19 Triplet...
Writing Epoch 20 Triplet...
Writing Epoch 21 Triplet...
Writing Epoch 22 Triplet...
Writing Epoch 23 Triplet...
Writing Epoch 24 Triplet...
Writing Epoch 25 Triplet...
Writing Epoch 26 Triplet...
Writing Epoch 27 Triplet...
Writing Epoch 28 Triplet...
Writing Epoch 29 Triplet...
Writing Epoch 30 Triplet...
Cost time 0:28:36.327094


In [ ]:
# Create validation set divided with labels (No use)
import os
import shutil
# Load validation labels
file2 = open("tiny-imagenet-200/val/val_annotations.txt", "r")
valpic = []
vallabel = []
tt = file2.readlines()
for line in tt:
    lines = re.split("\t",line)
    valpic.append(lines[0])
    vallabel.append(lines[1])
file2.close()
# Create directories and move pictures
valfolder = "tiny-imagenet-200/val/images/"
alllabel = list(set(vallabel))
for label in alllabel:
    os.makedirs(valfolder+label)
for (pic,label) in zip(valpic,vallabel):
    shutil.move(valfolder+pic, valfolder+label)

In [ ]:
# In validation set, transform all grey images to "RGB", all image sizes to (224 x 224 x 3), and save (No Use)
import os
import torchvision.transforms as transforms

start_time = datetime.datetime.now()

valfolder = "tiny-imagenet-200/val/images/"
trans_rz = transforms.Resize(size=(224,224))

valall = os.listdir(valfolder)
for i, vfold in enumerate(valall):
    valcls = os.listdir(os.path.join(valfolder,vfold))
    for image in valcls:
        picpath = os.path.join(valfolder, vfold, image)
        pic = Image.open(picpath)
        if pic.mode == "L":
            pic = pic.convert("RGB")
        pic = trans_rz(pic)
        pic.save(picpath,"JPEG") # Then save the new pic with the same name
    if (i+1)%20==0:
        print("{} classes of pictures have been transformed...".format(i+1))
        now_time = datetime.datetime.now()
        print("Cost time: {}".format(now_time-start_time))

In [ ]:
# Similarly for training set, but will take 10 times longer than validation set (No use)
import os
import torchvision.transforms as transforms

start_time = datetime.datetime.now()

trainfolder = "tiny-imagenet-200/train"
trainall = os.listdir(trainfolder)[170:]
trans_rz = transforms.Resize(size=(224,224))

for j, tfold in enumerate(trainall):
    traincls = os.listdir(os.path.join(trainfolder, tfold, "Images"))
    for image in traincls:
        picpath = os.path.join(trainfolder, tfold, "Images", image)
        pic = Image.open(picpath)
        if pic.mode == "L":
            pic = pic.convert("RGB")
        pic = trans_rz(pic)
        pic.save(picpath, "JPEG") # Then save the new pic with the same name
    if (j+1)%10==0:
        print("{} classes of pictures have been transformed...".format(j+1))
        now_time = datetime.datetime.now()
        print("Cost time: {}".format(now_time-start_time))

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_tp = 200 # Number of classes
num_im = 500 # Number of images in each class
total_im = num_tp * num_im
num_epochs = 30
batch_size = 100
learning_rate = 1e-3
embedding_size = 512

In [3]:
import torchvision.transforms as transforms

# Load validation set
valfolder = "tiny-imagenet-200/val/images/"
valdataset = torchvision.datasets.ImageFolder(valfolder, transform=transforms.ToTensor()) 
val_loader = Data.DataLoader(valdataset, batch_size=1, shuffle=False, num_workers=4)

# Load training set for convenience of embedding
trainfolder = "tiny-imagenet-200/train"
train_embed_dataset = torchvision.datasets.ImageFolder(trainfolder, transform=transforms.ToTensor()) 
train_embed_loader = Data.DataLoader(train_embed_dataset, batch_size=1, shuffle=False, num_workers=4)

In [4]:
# Build model
# Problem: How to substitute the last linear layer so that embedding vector instead of classification is returned
# Define pre-trained model ResNet18
def resnet18(pretrained=True):
    model = torchvision.models.resnet.ResNet(torchvision.models.resnet.BasicBlock,[2,2,2,2])
    if pretrained:
        model.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/resnet18-5c106cde.pth',model_dir="./"))
    return model

mod = resnet18()
model = copy.deepcopy(mod)
model.fc = nn.Linear(512, embedding_size, bias=True) # Adapt for our model
model = model.to(device)

# Build criterion and optimizer
criterion = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
class TinyImageDataset(Data.Dataset):
    def __init__(self, impath, label=None, train=True):
        self.path = impath
        self.label = label
        self.train = train
    
    def __getitem__(self, index):
        """
        Args:
        index (int): Index of Dataset
        
        Return:
        if train: 
            img_triplet (np.array, 3 x 224 x 224 x 3)
        else: (Will return error if label==None and train==False)
            img (np.array, 224 x 224 x 3), label (str) 
        """
        if self.train:
            img_triplet_path = re.split(string=self.path[index], pattern="\t")
            img_triplet_path = [re.sub("HW5/","",img_triplet_path[i]) for i in range(3)]
            img_triplet = [np.reshape(io.imread(img_triplet_path[i]),newshape=(3,224,224)) for i in range(3)]
            return img_triplet
        else:
            img = io.imread(self.path[index])
            label = self.label[index]
            return img, label
    
    def __len__(self):
        return len(self.path)
    
    def __repr__(self):
        return "Triplet_Generate_TinyImageNet200"

In [ ]:
# In each epoch, zip a new trainset and train the network
for epoch in range(num_epochs):
    # Load new dataset
    file1 = open("ImagePath\\Epoch_{}_Triplet.txt".format(epoch+1), "r")
    tm = file1.readlines()
    train_dataset = TinyImageDataset(impath = tm)
    train_loader = Data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # Run model
    total = 0
    correct = 0 # Count accuracy in each epoch
    loss_total = 0 # Total loss in each epoch
    for i, image_triplet in enumerate(train_loader):
        image_qr = image_triplet[0].to(device)
        image_ps = image_triplet[1].to(device)
        image_ng = image_triplet[2].to(device)
            
        # Forward
        embed_qr = model(image_qr)
        embed_ps = model(image_ps)
        embed_ng = model(image_ng)
        loss = criterion(embed_qr, embed_ps, embed_ng)
        total += image_qr.size(0)
        correct += (loss==0).sum().item()
        loss_total += loss.item()
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch[{}/{}], Total Loss {:4f}, Accuracy {:4f}%".format(
            epoch+1, num_epochs, loss_total, correct/total*100))
    
    # Compute test accuracy on validation set
    if (epoch+1)%5 == 0: # Do testing on validation set
        train_embed = np.empty([total_im, embedding_size])
        # First, compute training embeddings
        for i, (image,label) in enumerate(train_embed_loader):
            image_embed = model(image)
            train_embed[i,:] = image_embed
        # Then, for each validation image
        count_correct = 0
        totalcount = len(val_loader)
        for j, (image,label) in enumerate(val_loader):
            image_embed = model(image)
            image_exp_embed = np.tile(image_embed, reps=(total_im,1))
            
            dist_embed = train_embed - image_exp_embed
            dist = np.sum(dist_embed**2, axis=1, keepdims=True)
            argdistidx = np.argsort(dist)[:30]
            argdistlabel = argdistidx // num_im
            arglabel = max(set(argdistlabel), key=argdistlabel.count)
            if arglabel == label:
                count_correct += 1
        print("Validation accuracy in {} epoch: {}%".format(epoch+1, count_correct/totalcount * 100))

In [ ]:
# In each epoch, zip a new trainset and train the network
# This version has been abandoned. I have chosen to preprocess the images instead.
for epoch in num_epochs:
    file1 = open("ImagePath\\Epoch_{}_Triplet.txt".format(epoch+1), "r")
    tm = file1.readlines()
    im_qr_arr = np.zeros([1,224,224,3])
    im_ps_arr = np.zeros([1,224,224,3])
    im_ng_arr = np.zeros([1,224,224,3])
    # Zip image arrays as Tensors
    for idx in range(num_tp*num_im):
        tmp_triplet = tm[idx]
        tmp_path = re.split(string=tmp_triplet, pattern="\t")
        im_qr = io.imread(tmp_path[0])
        im_qr = grey2rgb(im_qr)
        im_ps = io.imread(tmp_path[1])
        im_ps = grey2rgb(im_ps)
        im_ng = io.imread(tmp_path[2])
        im_ng = grey2rgb(im_ng)
        # Do transforms on these images
        # Website: http://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html
        im_qr = tf.resize(im_qr,output_shape=(224,224))
        im_ps = tf.resize(im_ps,output_shape=(224,224))
        im_ng = tf.resize(im_ng,output_shape=(224,224))
        # Concat each image to corresponding Tensors
        im_qr_arr = np.concatenate((im_qr_arr, np.expand_dims(im_qr, axis=0)))
        im_ps_arr = np.concatenate((im_ps_arr, np.expand_dims(im_ps, axis=0)))
        im_ng_arr = np.concatenate((im_ng_arr, np.expand_dims(im_ng, axis=0)))
    tm.close()
    # Zip as Tensor
    im_qr_arr = im_qr_arr[1:,:,:,:]
    im_ps_arr = im_ps_arr[1:,:,:,:]
    im_ng_arr = im_ng_arr[1:,:,:,:]
    im_qr_tensor = torch.Tensor(im_qr_arr)
    im_ps_tensor = torch.Tensor(im_ps_arr)
    im_ng_tensor = torch.Tensor(im_ng_arr)
    # Zip images in a Data.Dataloader
    trainset = Data.TensorDataset(im_qr_tensor, im_ps_tensor, im_ng_tensor)
    train_loader = Data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # Run model
    total = 0
    correct = 0 # Count accuracy in each epoch
    loss_total = 0 # 
    for i,(image_qr, image_ps, image_ng) in enumerate(train_loader):
        image_qr = image_qr.to(device)
        image_ps = image_ps.to(device)
        image_ng = image_ng.to(device)
            
        # Forward
        embed_qr = model(image_qr)
        embed_ps = model(image_ps)
        embed_ng = model(image_ng)
        loss = criterion(embed_qr, embed_ps, embed_ng)
        total += image_qr.size(0)
        correct += (loss==0).sum().item()
        loss_total += loss.item()
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch[{}/{}], Total Loss {:4f}, Accuracy {:4f}%".format(
            epoch+1, num_epochs, loss_total, correct/total*100))